<a href="https://colab.research.google.com/github/skhazaei/TensorFlow-repo/blob/master/diabetes_dataset_logs_dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check [here](https://keras.io/callbacks/) for custom callbacks

In [1]:
from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()
diabetes_dataset.keys()

dict_keys(['data', 'target', 'DESCR', 'feature_names', 'data_filename', 'target_filename'])

In [2]:
data = diabetes_dataset['data']
target = diabetes_dataset['target']

In [3]:
target = (target - target.mean(axis=0)) / target.std()

In [4]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.1)

In [5]:
print(f'train data shape: {train_data.shape}')
print(f'test data shape: {test_data.shape}')
print(f'train target shape: {train_target.shape}')
print(f'test target shape: {test_target.shape}')

train data shape: (397, 10)
test data shape: (45, 10)
train target shape: (397,)
test target shape: (45,)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1408      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 18,305
Trainable params: 18,177
Non-trainable params: 128
__________________________________________________

In [8]:
model.compile(loss='mse', optimizer="adam", metrics=['mae'])

# Let's create a custom callback

In [9]:
class LossAndMetricCallback(tf.keras.callbacks.Callback):

    # Print the loss every two steps of training
    def on_train_batch_end(self, batch, logs=None):
        if batch %2 ==0:
          print('After batch {}, the loss is {}'.format(batch, logs['loss']))
    
    def on_epoch_begin(self, epoch, logs=None):
      print('Start of epoch {}'.format(epoch+1))
    
    # Print the loss after each batch in the test set
    def on_test_batch_end(self, batch, logs=None):
      print('After batch {}, the loss is {}'.format(batch, logs['loss']))

    # Print the loss and mean absolute error after each epoch
    def on_epoch_end(self, epoch, logs=None):
      print('End of epoch {}: Average loss is {}, and mean absolute error is {}'.format(epoch+1, logs['loss'], logs['mae']))
    
    # Notify the user when prediction has finished on each batch
    def on_predict_batch_end(self,batch, logs=None):
        print("Finished prediction on batch {}!".format(batch))

In [10]:
history = model.fit(train_data, train_target, epochs=2,
                    batch_size=64, callbacks=[LossAndMetricCallback()], verbose=False)

Start of epoch 1
After batch 0, the loss is 1.0404589176177979
After batch 2, the loss is 0.9033668637275696
After batch 4, the loss is 0.870500922203064
After batch 6, the loss is 0.8140500783920288
End of epoch 1: Average loss is 0.8140500783920288, and mean absolute error is 0.7629238367080688
Start of epoch 2
After batch 0, the loss is 0.6351763606071472
After batch 2, the loss is 0.554715096950531
After batch 4, the loss is 0.5279847383499146
After batch 6, the loss is 0.5434674024581909
End of epoch 2: Average loss is 0.5434674024581909, and mean absolute error is 0.6010139584541321


In [11]:
model_eval = model.evaluate(test_data, test_target, batch_size=10, 
                            callbacks=[LossAndMetricCallback()], verbose=False)

After batch 0, the loss is 0.9949547648429871
After batch 1, the loss is 0.7982174158096313
After batch 2, the loss is 0.7012218236923218
After batch 3, the loss is 0.8305608034133911
After batch 4, the loss is 0.8850511312484741


In [12]:
model_pred = model.predict(test_data, batch_size=10,
                           callbacks=[LossAndMetricCallback()], verbose=False)

Finished prediction on batch 0!
Finished prediction on batch 1!
Finished prediction on batch 2!
Finished prediction on batch 3!
Finished prediction on batch 4!


# A more sophisticated custom callback: Learning rate scheduler 

To define a callback to change the learning rate of the optimiser of a model during training. We will do this by specifying the epochs and new learning rates where we would like it to be changed.





In [13]:
# Define the learning rate schedule. The tuples below are (start_epoch, new_learning_rate)
lr_schedule = [(4, 0.03), (7, 0.02), (11, 0.005), (15, 0.007)]


def get_new_epoch_lr(epoch, lr):
    # Checks to see if the input epoch is listed in the learning rate schedule 
    # and if so, returns index in lr_schedule
    epoch_in_sched = [i for i in range(len(lr_schedule)) if lr_schedule[i][0]==int(epoch)]
    if len(epoch_in_sched)>0:
        new_lr = lr_schedule[epoch_in_sched[0]][1]      
        # If it is, return the learning rate corresponding to the epoch
        return new_lr
    else:
        # Otherwise, return the existing learning rate
        return lr

In [14]:
class LRScheduler(tf.keras.callbacks.Callback):
    
    def __init__(self, new_lr):
        super(LRScheduler, self).__init__()
        # Add the new learning rate function to our callback
        self.new_lr = new_lr

    def on_epoch_begin(self, epoch, logs=None):
        # Make sure that the optimizer we have chosen has a learning rate, and raise an error if not
        if not hasattr(self.model.optimizer, 'lr'):
              raise ValueError('Error: Optimizer does not have a learning rate.')
                
        # Get the current learning rate
        curr_rate = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        
        # Call the auxillary function to get the scheduled learning rate for the current epoch
        scheduled_rate = self.new_lr(epoch, curr_rate)

        # Set the learning rate to the scheduled learning rate
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_rate)
        print('Learning rate for epoch {} is {:7.3f}'.format(epoch, scheduled_rate))

In [15]:
new_model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])

In [16]:
new_model.compile(loss='mse',
                optimizer="adam",
                metrics=['mae', 'mse'])

In [17]:
new_history = new_model.fit(train_data, train_target, epochs=20,
                            batch_size=100, callbacks=[LRScheduler(get_new_epoch_lr)], verbose=False)

Learning rate for epoch 0 is   0.001
Learning rate for epoch 1 is   0.001
Learning rate for epoch 2 is   0.001
Learning rate for epoch 3 is   0.001
Learning rate for epoch 4 is   0.030
Learning rate for epoch 5 is   0.030
Learning rate for epoch 6 is   0.030
Learning rate for epoch 7 is   0.020
Learning rate for epoch 8 is   0.020
Learning rate for epoch 9 is   0.020
Learning rate for epoch 10 is   0.020
Learning rate for epoch 11 is   0.005
Learning rate for epoch 12 is   0.005
Learning rate for epoch 13 is   0.005
Learning rate for epoch 14 is   0.005
Learning rate for epoch 15 is   0.007
Learning rate for epoch 16 is   0.007
Learning rate for epoch 17 is   0.007
Learning rate for epoch 18 is   0.007
Learning rate for epoch 19 is   0.007
